## 한글 topic modeling

In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
import os
from pathlib import Path

In [2]:
# df_read = pd.read_excel('Scraped Data processed\\04_news\\NEWS_chosun01_cmt.xlsx', sheet_name='chosun01')
df_read = pd.read_excel('NEWS_chosun_cmt_total.xlsx')

In [3]:
df_total = df_read.copy()

In [4]:
df_total['cmt_text'] = df_total['cmt_text'].apply(str) #text에 숫자가 들어 있는 것이 있어서 CountVectorizer할 때 error가 생기므로 먼저 string처리

In [5]:
df_total['cmt_processed'] = df_total['cmt_text']

In [6]:
len(df_total)

100

#### konlpy를 사용한 token화

In [7]:
from konlpy.tag import Okt  
okt=Okt() 

In [8]:
def noun_text(text):
    results = okt.pos(text, norm=True, stem=True)
    noun = [word for word, pos in results if pos == "Noun"]
    return noun

In [9]:
df_total['cmt_noun'] = df_total['cmt_processed'].apply(noun_text)

#### 불용어 제거

In [10]:
def stopwords_process(text):
    #https://www.ranks.nl/stopwords/korean
    stop_words = ['아', '휴', '어', '나', '의해', '따라', '을', '를', '에', '의', '가', '으로', '로', '에게', '저', '다', '하여야',
                 '도', '하지만', '이리하여', '그리하여', '로써', '으로써', '등', '등등', '제', '여', '남짓', '몇', '또한', '면',
                 '자면', '다면', '툭', '딱', '각', '와', '과', '그러므로', '그래서', '고로', '거니와', '이지만', '가', '하', 
                 '와', '오', '왜', '야', '흥', '휴', '앗', '면', '답다', '및', '즉', '몇', '만큼', '좀', '곧', '그래', '그렇지',
                  '즉', '허', '허걱', '팍', '퍽', '에서', '해요', '습니다', '입니다', '이다', '것', '매', '들', '모', '저기',
                  '저쪽', '그럼', '그러면', '그래', '그때', '그저', '저것만큼', '그', '하면서', '같다', '있다', '없다', '와같이',
                  '무엇', '무슨', '어느', '어떤', '고', '때', '네', '예', '누구', '그러니', '그러니까', '그', '너희', '저희', '우리',
                  '너', '위하여', '나', '오호', '아하', '어쨌든', '흐흐', '자면', '마치', '아니라면', '쉿', '하든지', '이라면',
                  '하나', '일', '이 되다', '그래서', '다', '운운', '이러이러하다', '에 있다', '하기는한데', '어떻게', '자', '이',
                  '이것', '여기', '이러한', '이와 같은', '이만큼', '이때', '하고 있다', '한 후', '자기', '같이', '대로 하다', '으로서',
                  '참', '년', '월', '일', '령', '영', "", '은', '만', '인', '동', '관리자', '댓글', '삭제', '히', '수', '니', '으', '우',
                 '하다', '되다']
    without_stopwords =[word for word in text if not word in stop_words]
    return without_stopwords

In [11]:
df_total['cmt_noun'] = df_total['cmt_noun'].apply(stopwords_process)

## gensim으로 topic modeling (Total) 

In [12]:
from gensim import corpora

dictionary_noun = corpora.Dictionary(df_total['cmt_noun'])
corpus_noun = [dictionary_noun.doc2bow(text) for text in df_total['cmt_noun']]

In [20]:
import gensim
    
NUM_TOPICS = 6 #20개의 토픽, k=20
print("noun")
ldamodel = gensim.models.ldamodel.LdaModel(corpus_noun, num_topics = NUM_TOPICS, id2word=dictionary_noun, passes=15)
topics = ldamodel.print_topics(num_words=6) # 토픽 당 보여줄 단어 수
for topic in topics:
    print(topic)

noun
(0, '0.023*"조선" + 0.016*"일본" + 0.010*"나라" + 0.007*"역사" + 0.007*"공부" + 0.007*"말"')
(1, '0.017*"글" + 0.013*"역사" + 0.012*"세기" + 0.012*"일본" + 0.010*"사용" + 0.009*"나라"')
(2, '0.029*"우리나라" + 0.022*"미국" + 0.022*"일본" + 0.015*"위" + 0.015*"독립" + 0.015*"개"')
(3, '0.015*"생각" + 0.015*"일본" + 0.011*"조선" + 0.009*"사실" + 0.007*"역사" + 0.007*"지도자"')
(4, '0.013*"나라" + 0.013*"미국" + 0.010*"일본" + 0.008*"어찌" + 0.008*"친일파" + 0.008*"중국"')
(5, '0.028*"역사" + 0.012*"조총" + 0.008*"위키" + 0.008*"나무" + 0.008*"종인" + 0.008*"박"')


각 단어 앞에 붙은 수치는 단어의 해당 토픽에 대한 기여도를 보여줍니다. 또한 맨 앞에 있는 토픽 번호는 0부터 시작하므로 총 20개의 토픽은 0부터 19까지의 번호가 할당되어져 있습니다. passes는 알고리즘의 동작 횟수를 말하는데, 알고리즘이 결정하는 토픽의 값이 적절히 수렴할 수 있도록 충분히 적당한 횟수를 정해주면 됩니다. 여기서는 총 15회를 수행하였습니다. 여기서는 num_words=4로 총 4개의 단어만 출력하도록 하였습니다. 만약 10개의 단어를 출력하고 싶다면 아래의 코드를 수행하면 됩니다.

In [21]:
import pyLDAvis
import pyLDAvis.gensim

pyLDAvis.enable_notebook()
#vis = pyLDAvis.gensim.prepare(ldamodel, corpus, dictionary=lda_model.id2word)
vis = pyLDAvis.gensim.prepare(ldamodel, corpus_noun, dictionary_noun, sort_topics = False)
pyLDAvis.display(vis)
#아래 오른쪽에서 각 막대별로 의미하는 단어가 출력되어야 하는데 안 되네?

## 문서 별 토픽 분포 보기

In [15]:
for i, topic_list in enumerate(ldamodel[corpus_noun]):
    if i==5:
        break
    print(i,'번째 문서의 topic 비율은',topic_list)

0 번째 문서의 topic 비율은 [(3, 0.97806466)]
1 번째 문서의 topic 비율은 [(0, 0.25), (1, 0.25), (2, 0.25), (3, 0.25)]
2 번째 문서의 topic 비율은 [(0, 0.0132425465), (1, 0.013229531), (2, 0.9603392), (3, 0.013188689)]
3 번째 문서의 topic 비율은 [(0, 0.033345353), (1, 0.90172404), (2, 0.031787187), (3, 0.033143446)]
4 번째 문서의 topic 비율은 [(0, 0.94893706), (1, 0.017129833), (2, 0.01693539), (3, 0.016997768)]


In [16]:
def make_topictable_per_doc(ldamodel, corpus):
    topic_table = pd.DataFrame()

    # 몇 번째 문서인지를 의미하는 문서 번호와 해당 문서의 토픽 비중을 한 줄씩 꺼내온다.
    for i, topic_list in enumerate(ldamodel[corpus]):
        doc = topic_list[0] if ldamodel.per_word_topics else topic_list            
        doc = sorted(doc, key=lambda x: (x[1]), reverse=True)
        # 각 문서에 대해서 비중이 높은 토픽순으로 토픽을 정렬한다.
        # EX) 정렬 전 0번 문서 : (2번 토픽, 48.5%), (8번 토픽, 25%), (10번 토픽, 5%), (12번 토픽, 21.5%), 
        # Ex) 정렬 후 0번 문서 : (2번 토픽, 48.5%), (8번 토픽, 25%), (12번 토픽, 21.5%), (10번 토픽, 5%)
        # 48 > 25 > 21 > 5 순으로 정렬이 된 것.

        # 모든 문서에 대해서 각각 아래를 수행
        for j, (topic_num, prop_topic) in enumerate(doc): #  몇 번 토픽인지와 비중을 나눠서 저장한다.
            if j == 0:  # 정렬을 한 상태이므로 가장 앞에 있는 것이 가장 비중이 높은 토픽
                topic_table = topic_table.append(pd.Series([int(topic_num), round(prop_topic,4), topic_list]), ignore_index=True)
                # 가장 비중이 높은 토픽과, 가장 비중이 높은 토픽의 비중과, 전체 토픽의 비중을 저장한다.
            else:
                break
    return(topic_table)

In [17]:
topictable = make_topictable_per_doc(ldamodel, corpus_noun)
topictable = topictable.reset_index() # 문서 번호을 의미하는 열(column)로 사용하기 위해서 인덱스 열을 하나 더 만든다.
topictable.columns = ['문서 번호', '가장 비중이 높은 토픽', '가장 높은 토픽의 비중', '각 토픽의 비중']
topictable[:10]

,문서 번호,가장 비중이 높은 토픽,가장 높은 토픽의 비중,각 토픽의 비중
0,0,3.0,0.9781,"[(3, 0.97806376)]"
1,1,0.0,0.2500,"[(0, 0.25), (1, 0.25), (2, 0.25), (3, 0.25)]"
2,2,2.0,0.9603,"[(0, 0.013242604), (1, 0.013229525), (2, 0.960..."
3,3,1.0,0.9017,"[(0, 0.033334404), (1, 0.90173864), (2, 0.0317..."
4,4,0.0,0.9489,"[(0, 0.9489376), (1, 0.01712941), (2, 0.016935..."
5,5,0.0,0.9631,"[(0, 0.96305794), (1, 0.012193335), (2, 0.0123..."
6,6,0.0,0.9577,"[(0, 0.957661), (1, 0.013996983), (2, 0.014323..."
7,7,0.0,0.9819,"[(0, 0.9818989)]"
8,8,0.0,0.9891,"[(0, 0.98911804)]"
9,9,0.0,0.9883,"[(0, 0.98834735)]"
